# Proyecto

**Objetivos**:

* Realizar un análisis exploratorio de datos (EDA) con la finalidad de mejor la distribución de los datos utilizados (en este caso, las caracteristicas de los picos de los fringílidos a lo largo de los años y de la información hereditaria), como estan clasificados, su distribución, media, quartiles, etc.

* Determinar mediante una prueba de hipótesis si realmente hubo un cambio entre las caracteristicas del pico de estos animales a lo largo de los años.

* Adicional al punto anterior, tratar de determinar si existe una relación entre el las características físicas de los padres y aquellas de las crías.

In [1]:
# Librerias utiles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

## Sobre el conjunto de Datos

Durante los últimos más de 40 años (de 1973 a 2013), Peter y Rosemary Grant visitaron la isla Daphne Major de Galápagos y recopilaron datos sobre los pinzones de Darwin para estudiar la teoría de la evolución de Darwin.

Los Grant pasan años observando, marcando y midiendo los pinzones de Darwin y su entorno. Durante ese tiempo documentaron cambios ambientales y cómo estos cambios favorecieron a ciertos individuos dentro de la población, ilustrando la selección natural en acción.

El conjunto de datos se centra en la dimensión del pico de los pinzones. Para los pinzones, el tamaño y la forma de sus picos son rasgos que varían para adaptarse a los cambios ambientales. La relación entre el tamaño del pico y los eventos ecológicos que se ha popularizado un poco como un ejemplo de "evolución que ocurre ante nuestros ojos".



In [3]:
df_1975 = pd.read_csv('./data/finch_beaks_1975.csv')
df_2012 = pd.read_csv('./data/finch_beaks_2012.csv')

df_fortis = pd.read_csv('./data/fortis_beak_depth_heredity.csv')
df_scandens = pd.read_csv('./data/scandens_beak_depth_heredity.csv')

Comenzamos por hacer un procesamiento de los datos y hacer cálculos de estadística descriptiva junto con un Análisis Exploratorio de Datos.

Tenemos en total 4 conjuntos de Datos
- Finch Beak 1975: La medida del pico de los pinzones registrada en 1975.
- Finch Beak 2012: La medida del pico de los pinzones registrada en 2012.
- Fortis Beak Depth Heredity: Información hereditaria del pico de G. fortis.
- Scandens Beak Depth Heredity: Información hereditaria del pico de G. scandens.

In [3]:
df_1975.info() # No hay valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   band             403 non-null    int64  
 1   species          403 non-null    object 
 2   Beak length, mm  403 non-null    float64
 3   Beak depth, mm   403 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 12.7+ KB


In [4]:
df_2012.info() # Tampoco hay valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   band     248 non-null    int64  
 1   species  248 non-null    object 
 2   blength  248 non-null    float64
 3   bdepth   248 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 7.9+ KB


In [5]:
df_fortis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Mid-offspr  413 non-null    float64
 1   Male BD     413 non-null    float64
 2   Female BD   413 non-null    float64
dtypes: float64(3)
memory usage: 9.8 KB


In [6]:
df_scandens.info() # Tampoco

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mid_parent     130 non-null    float64
 1   mid_offspring  130 non-null    float64
dtypes: float64(2)
memory usage: 2.2 KB


## PARTE JOSE

Tras ver la información de los datos, vemos que no existen datos faltantes. Además que la mayoría de las variables son numéricas, exceptuando por las especies.

Si observamos los valores de la columna species...

In [4]:
# Vemos que los valores de la columna species son fortis y scandens en ambos dataframes
print(df_1975['species'].unique()) 
print(df_2012['species'].unique())

['fortis' 'scandens']
['fortis' 'scandens']



Otra cosa que hay que observar es que el nombre de las columnas no son iguales (tanto en el dataframe de 1975 como en el de 2012). Renombraremos las columnas para mantener la consistencia entre los dos dataframes


In [5]:
# Renombrando las columnas para mantener consistencia entre los nombres

df_1975.rename(
    columns={'Beak length, mm': 'beak_length', 'Beak depth, mm': 'beak_depth'}, 
    inplace=True
)

df_2012.rename(
    columns={'blength': 'beak_length', 'bdepth': 'beak_depth'},
    inplace=True
)

## PARTE JESUS

## ¿Existe realmente una diferencia entre las aves de 1975 y las de 2012?

Uno de los objetivos presentados en este proyecto fue el de determinar si en realidad había una diferencia entre las caracteristicas de los picos de los fringílidos. Para esto podemos realizar una prueba de hipótesis, para tratar de determinar si la media tanto de la longitud como de la profundad de los picos en estas especies es diferente entre los años.

Sin embargo, como nuestro dataset incluye muestras de dos diferentes especies, es necesario separarlas de manera correspondiente.

In [7]:
# Separamos para la especie scandens
df_scandens_1975 = df_1975[df_1975['species'] == 'scandens']
df_scandens_2012 = df_2012[df_2012['species'] == 'scandens']

print(f"Tamaño de muestra G. scandens en 1975: {df_scandens_1975.shape[0]}")
print(f"Tamaño de muestra G. scandens en 2012: {df_scandens_2012.shape[0]}")

Tamaño de muestra G. scandens en 1975: 87
Tamaño de muestra G. scandens en 2012: 127


In [8]:
df_fortis_1975 = df_1975[df_1975['species'] == 'fortis']
df_fortis_2012 = df_2012[df_2012['species'] == 'fortis']

print(f"Tamaño de muestra G. fortis en 1975: {df_fortis_1975.shape[0]}")
print(f"Tamaño de muestra G. fortis en 2012: {df_fortis_2012.shape[0]}")

Tamaño de muestra G. fortis en 1975: 316
Tamaño de muestra G. fortis en 2012: 121


Una vez separadas nuestras muestras, podemos tratar de observar si existe alguna diferencia entre el promedio de éstas.

In [9]:
print(f"Longitud promedio del pico para G. fortis en 1975: {df_fortis_1975['Beak length, mm'].mean():.4f} mm")
print(f"Longitud promedio del pico para G. fortis en 2012: {df_fortis_2012['blength'].mean():.4f} mm")

Longitud promedio del pico para G. fortis en 1975: 10.5652 mm
Longitud promedio del pico para G. fortis en 2012: 10.5174 mm


In [10]:
print(f"Profundidad promedio del pico para G. fortis en 1975: {df_fortis_1975['Beak depth, mm'].mean():.4f} mm")
print(f"Profundidad promedio del pico para G. fortis en 2025: {df_fortis_2012['bdepth'].mean():.4f} mm")

Profundidad promedio del pico para G. fortis en 1975: 9.1716 mm
Profundidad promedio del pico para G. fortis en 2025: 8.6054 mm


A simple vista, puede parecer que tanto la longitud como la profundidad del pico de la especie *G. fortis* parecen haberse encogido, pero para determinar si hay una verdadera diferencia entre las aves en ambos años, podemos plantearnos la siguiente prueba de hipótesis.

$$ 
\begin{matrix}
H_0: & \mu_0 - \mu_1 = 0\\
H_1: & \mu_0 - \mu_1 > 0
\end{matrix}
$$

Donde $\mu_0$ y $\mu_1$ se refieren a la característica promedio de la especie en 1975 y en 2012, respectivamente. Aquí, la hipótesis nula indica que no existe una diferencia entre la característica promedio, es decir que $\mu_0 = \mu_1$, y la hipótesis alternativa propone que la característica del pico se ha encogido con el tiempo, o bien que $\mu_0 > \mu_1$, todo esto para un nivel de confiaza $\alpha = 0.05$.

En los casos donde comparamos dos medias poblacionales, el estadístico de prueba está dado por:

$$
z = \frac{\bar{X} - \bar{Y} - \Delta_0}{\sqrt{\frac{\sigma_{0}^{2}}{m} + \frac{\sigma_{1}^{2}}{n} }}
$$

Donde $\Delta_0$ hace referencia a la diferencia de medias buscada, la cual en nuestro caso es igual a 0, lo que reduce la expresión a:

$$
z = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{\sigma_{0}^{2}}{m} + \frac{\sigma_{1}^{2}}{n} }}
$$

Podemos comenzar a definir una funcion para obtener el valor de este estadistico:

In [26]:
def calcular_z(series1,series2):
    """
    Obtener el valor de z a partir de dos series de pandas
    """
    diff_prom = series1.mean() - series2.mean()
    
    valor1_raiz = (series1.var() / series1.size)
    valor2_raiz = (series2.var() / series2.size)
    
    z = diff_prom / np.sqrt(valor1_raiz + valor2_raiz)
    
    return z

In [19]:
z_long_fortis = calcular_z(df_fortis_1975['Beak length, mm'],df_fortis_2012['blength'])
z_prof_fortis = calcular_z(df_fortis_1975['Beak depth, mm'],df_fortis_2012['bdepth'])

print(z_long_fortis)
print(z_prof_fortis)

0.5791225584108269
7.217487552951728


In [33]:
z_critico = stats.norm.ppf(0.05)
z_critico

-1.6448536269514729